In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastai

from sklearn import metrics
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor , RandomForestClassifier
import re
import seaborn as sns

from sklearn import metrics
import io
import xgboost as xgb


In [4]:
PATH = './data/'
!ls {PATH}

sample_submission.csv  test.csv  train.csv


In [5]:
train_raw = pd.read_csv(f'{PATH}train.csv',low_memory=False,parse_dates=['opened_at'])
test_raw = pd.read_csv(f'{PATH}test.csv',low_memory=False,parse_dates=['opened_at'])

In [6]:
def extracting_num(df_raw):
  df = df_raw.copy()
  df['opened_by'] = df['opened_by'].str.split(expand=True)[2]
  df['location'] = df['location'].str.split(expand=True)[1]
  df['category'] = df['category'].str.split(expand=True)[1]
  df['subcategory'] = df['subcategory'].str.split(expand=True)[1]
  df['impact'] = df['impact'].str.split(expand=True)[0]
  df['urgency'] = df['urgency'].str.split(expand=True)[0]
  df['priority'] = df['priority'].str.split(expand=True)[0]
  df['assigned_to'] = df['assigned_to'].str.split(expand=True)[1]
  return df

In [7]:
train_raw = extracting_num(train_raw)

In [19]:
train_raw['urgency'].value_counts()

2    19117
3      557
1      326
Name: urgency, dtype: int64

In [25]:
for index,row in train_raw.iterrows() :
    if(row['target_days']>10):
        train_raw.drop(index,inplace=True)
for index,row in train_raw.iterrows() :
    if(row['target_days']<=4):
        train_raw.drop(index,inplace=True)

In [26]:
train_raw['knowledge'].value_counts()

False    12269
True       861
Name: knowledge, dtype: int64

In [27]:
test_raw['knowledge'].value_counts()

False    4918
Name: knowledge, dtype: int64

In [22]:
test_raw['impact'].value_counts()

2 - Medium    4732
3 - Low        127
1 - High        59
Name: impact, dtype: int64

13     878
17     612
94     364
33     362
194    286
      ... 
93       1
136      1
48       1
5        1
40       1
Name: assigned_to, Length: 135, dtype: int64

17     228
40      40
108     39
131     38
24      38
8       30
397     29
386     26
55      22
62      20
180     17
301     16
305     15
221     15
500     14
58      13
239     12
501     10
240      9
390      8
94       5
444      5
512      4
443      3
536      3
433      3
47       3
79       3
412      2
381      2
128      2
20       2
463      2
10       2
168      2
257      2
124      2
255      1
520      1
109      1
325      1
59       1
247      1
193      1
517      1
354      1
226      1
174      1
320      1
503      1
382      1
44       1
70       1
Name: opened_by, dtype: int64

204    151
161    110
143     97
93      67
108     60
      ... 
236      1
44       1
107      1
197      1
160      1
Name: location, Length: 71, dtype: int64

26    105
42    104
53     93
46     73
23     38
9      35
37     35
57     34
61     30
32     27
20     24
24     23
40     18
51     17
44     15
35     11
13      8
19      8
43      6
34      5
28      4
8       3
41      2
7       2
38      2
45      1
4       1
56      1
30      1
47      1
22      1
17      1
25      1
Name: category, dtype: int64

174    231
223     99
175     56
275     29
164     27
      ... 
189      1
153      1
151      1
90       1
243      1
Name: subcategory, Length: 88, dtype: int64